In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, minmax_scale, MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout, Activation, Flatten
import warnings
warnings.filterwarnings("ignore")

In [2]:
astro = pd.read_csv("E:/Python Programs/Astro_EuroMillions.csv")
astro.head()

,Data,Day,Nakshatra,Pada-1,Pada-2,Pada-3,Pada-4,1st Num,2nd Num,3rd Num,4th Num,5th Num,1st Lucky Star,2nd Lucky Star
0,Train,Tuesday,Ashwini,NaN,NaN,NaN,3.0,7,15,18,46,49,10,12
1,Train,Tuesday,Bharani,NaN,NaN,NaN,NaN,7,15,18,46,49,10,12
2,Train,Tuesday,Krittika,NaN,NaN,NaN,NaN,7,15,18,46,49,10,12
3,Train,Tuesday,Rohini,NaN,NaN,NaN,NaN,7,15,18,46,49,10,12
4,Train,Tuesday,Mrigashirsha,NaN,NaN,NaN,NaN,7,15,18,46,49,10,12


In [3]:
astro.fillna(value={"Pada-1":0, "Pada-2":0, "Pada-3":0, "Pada-4":0},inplace=True)

In [4]:
astro["Pada-1"] = astro["Pada-1"].astype(dtype='int32')
astro["Pada-2"] = astro["Pada-2"].astype(dtype='int32')
astro["Pada-3"] = astro["Pada-3"].astype(dtype='int32')
astro["Pada-4"] = astro["Pada-4"].astype(dtype='int32')

In [5]:
encoding = LabelEncoder()
astro["Day"] = encoding.fit_transform(astro["Day"])
astro["Nakshatra"] = encoding.fit_transform(astro["Nakshatra"])

In [6]:
astro.head()

,Data,Day,Nakshatra,Pada-1,Pada-2,Pada-3,Pada-4,1st Num,2nd Num,3rd Num,4th Num,5th Num,1st Lucky Star,2nd Lucky Star
0,Train,1,3,0,0,0,3,7,15,18,46,49,10,12
1,Train,1,4,0,0,0,0,7,15,18,46,49,10,12
2,Train,1,9,0,0,0,0,7,15,18,46,49,10,12
3,Train,1,19,0,0,0,0,7,15,18,46,49,10,12
4,Train,1,11,0,0,0,0,7,15,18,46,49,10,12


In [7]:
astro_train = astro[astro['Data'] == 'Train']
astro_test = astro[astro['Data'] == 'Test']

In [8]:
astro_train.head()

,Data,Day,Nakshatra,Pada-1,Pada-2,Pada-3,Pada-4,1st Num,2nd Num,3rd Num,4th Num,5th Num,1st Lucky Star,2nd Lucky Star
0,Train,1,3,0,0,0,3,7,15,18,46,49,10,12
1,Train,1,4,0,0,0,0,7,15,18,46,49,10,12
2,Train,1,9,0,0,0,0,7,15,18,46,49,10,12
3,Train,1,19,0,0,0,0,7,15,18,46,49,10,12
4,Train,1,11,0,0,0,0,7,15,18,46,49,10,12


In [9]:
astro_train.drop(labels='Data', axis=1, inplace=True)

In [10]:
astro_train.head()

,Day,Nakshatra,Pada-1,Pada-2,Pada-3,Pada-4,1st Num,2nd Num,3rd Num,4th Num,5th Num,1st Lucky Star,2nd Lucky Star
0,1,3,0,0,0,3,7,15,18,46,49,10,12
1,1,4,0,0,0,0,7,15,18,46,49,10,12
2,1,9,0,0,0,0,7,15,18,46,49,10,12
3,1,19,0,0,0,0,7,15,18,46,49,10,12
4,1,11,0,0,0,0,7,15,18,46,49,10,12


In [11]:
astro_test.head()

,Data,Day,Nakshatra,Pada-1,Pada-2,Pada-3,Pada-4,1st Num,2nd Num,3rd Num,4th Num,5th Num,1st Lucky Star,2nd Lucky Star
966,Test,0,3,0,0,0,6,6,9,10,30,49,3,4
967,Test,0,4,0,1,0,0,6,9,10,30,49,3,4
968,Test,0,9,0,3,0,0,6,9,10,30,49,3,4
969,Test,0,19,0,0,0,0,6,9,10,30,49,3,4
970,Test,0,11,0,0,0,0,6,9,10,30,49,3,4


In [12]:
astro_test.drop(labels='Data', axis=1, inplace=True)

In [13]:
astro_test.head()

,Day,Nakshatra,Pada-1,Pada-2,Pada-3,Pada-4,1st Num,2nd Num,3rd Num,4th Num,5th Num,1st Lucky Star,2nd Lucky Star
966,0,3,0,0,0,6,6,9,10,30,49,3,4
967,0,4,0,1,0,0,6,9,10,30,49,3,4
968,0,9,0,3,0,0,6,9,10,30,49,3,4
969,0,19,0,0,0,0,6,9,10,30,49,3,4
970,0,11,0,0,0,0,6,9,10,30,49,3,4


In [14]:
astro_train_scaled = pd.DataFrame(minmax_scale(astro_train), columns=astro_train.columns)
astro_train_scaled.head()

,Day,Nakshatra,Pada-1,Pada-2,Pada-3,Pada-4,1st Num,2nd Num,3rd Num,4th Num,5th Num,1st Lucky Star,2nd Lucky Star
0,1.0,0.115385,0.0,0.0,0.0,0.333333,0.230769,0.428571,0.257143,0.944444,0.966667,1.0,1.0
1,1.0,0.153846,0.0,0.0,0.0,0.000000,0.230769,0.428571,0.257143,0.944444,0.966667,1.0,1.0
2,1.0,0.346154,0.0,0.0,0.0,0.000000,0.230769,0.428571,0.257143,0.944444,0.966667,1.0,1.0
3,1.0,0.730769,0.0,0.0,0.0,0.000000,0.230769,0.428571,0.257143,0.944444,0.966667,1.0,1.0
4,1.0,0.423077,0.0,0.0,0.0,0.000000,0.230769,0.428571,0.257143,0.944444,0.966667,1.0,1.0


In [15]:
astro_test_scaled = pd.DataFrame(minmax_scale(astro_test), columns=astro_test.columns)
astro_test_scaled.head()

,Day,Nakshatra,Pada-1,Pada-2,Pada-3,Pada-4,1st Num,2nd Num,3rd Num,4th Num,5th Num,1st Lucky Star,2nd Lucky Star
0,0.0,0.115385,0.0,0.000000,0.0,0.666667,0.121212,0.15625,0.088235,0.558824,0.969697,0.333333,0.111111
1,0.0,0.153846,0.0,0.111111,0.0,0.000000,0.121212,0.15625,0.088235,0.558824,0.969697,0.333333,0.111111
2,0.0,0.346154,0.0,0.333333,0.0,0.000000,0.121212,0.15625,0.088235,0.558824,0.969697,0.333333,0.111111
3,0.0,0.730769,0.0,0.000000,0.0,0.000000,0.121212,0.15625,0.088235,0.558824,0.969697,0.333333,0.111111
4,0.0,0.423077,0.0,0.000000,0.0,0.000000,0.121212,0.15625,0.088235,0.558824,0.969697,0.333333,0.111111


In [16]:
X_tr = astro_train_scaled.loc[:,"Day":"Pada-4"]
y_tr_1 = astro_train_scaled["1st Num"]
y_tr_2 = astro_train_scaled["2nd Num"]
y_tr_3 = astro_train_scaled["3rd Num"]
y_tr_4 = astro_train_scaled["4th Num"]
y_tr_5 = astro_train_scaled["5th Num"]
y_tr_Luc1 = astro_train_scaled["1st Lucky Star"]
y_tr_Luc2 = astro_train_scaled["2nd Lucky Star"]

In [17]:
X_tr.head()

,Day,Nakshatra,Pada-1,Pada-2,Pada-3,Pada-4
0,1.0,0.115385,0.0,0.0,0.0,0.333333
1,1.0,0.153846,0.0,0.0,0.0,0.000000
2,1.0,0.346154,0.0,0.0,0.0,0.000000
3,1.0,0.730769,0.0,0.0,0.0,0.000000
4,1.0,0.423077,0.0,0.0,0.0,0.000000


In [18]:
X_tst = astro_test_scaled.loc[:,"Day":"Pada-4"]
y_tst_1 = astro_test_scaled["1st Num"]
y_tst_2 = astro_test_scaled["2nd Num"]
y_tst_3 = astro_test_scaled["3rd Num"]
y_tst_4 = astro_test_scaled["4th Num"]
y_tst_5 = astro_test_scaled["5th Num"]
y_tst_Luc1 = astro_test_scaled["1st Lucky Star"]
y_tst_Luc2 = astro_test_scaled["2nd Lucky Star"]

In [19]:
X_tst.head()

,Day,Nakshatra,Pada-1,Pada-2,Pada-3,Pada-4
0,0.0,0.115385,0.0,0.000000,0.0,0.666667
1,0.0,0.153846,0.0,0.111111,0.0,0.000000
2,0.0,0.346154,0.0,0.333333,0.0,0.000000
3,0.0,0.730769,0.0,0.000000,0.0,0.000000
4,0.0,0.423077,0.0,0.000000,0.0,0.000000


In [20]:
X_tr.shape, X_tst.shape, y_tr_1.shape, y_tst_1.shape

((966, 6), (527, 6), (966,), (527,))

In [21]:
X_train = np.array(X_tr.values)
X_train=np.reshape(X_train,(966,6,1))
print(X_train.shape)
X_train

(966, 6, 1)


array([[[1.        ],
        [0.11538462],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.33333333]],

       [[1.        ],
        [0.15384615],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ]],

       [[1.        ],
        [0.34615385],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ]],

       ...,

       [[1.        ],
        [0.92307692],
        [1.        ],
        [0.        ],
        [0.        ],
        [0.        ]],

       [[1.        ],
        [0.69230769],
        [0.        ],
        [0.55555556],
        [0.        ],
        [0.        ]],

       [[1.        ],
        [0.69230769],
        [0.        ],
        [0.44444444],
        [0.        ],
        [0.        ]]])

In [22]:
y_train_1 = np.array(y_tr_1.values)
y_train_1=np.reshape(y_train_1,(966,1))

y_train_2 = np.array(y_tr_2.values)
y_train_2=np.reshape(y_tr_2,(966,1))

y_train_3 = np.array(y_tr_3.values)
y_train_3=np.reshape(y_train_3,(966,1))

y_train_4 = np.array(y_tr_4.values)
y_train_4=np.reshape(y_tr_4,(966,1))

y_train_5 = np.array(y_tr_5.values)
y_train_5=np.reshape(y_tr_5,(966,1))

y_train_luck1 = np.array(y_tr_Luc1.values)
y_train_luck1=np.reshape(y_tr_Luc1,(966,1))

y_train_luck2 = np.array(y_tr_Luc2.values)
y_train_luck2=np.reshape(y_tr_Luc2,(966,1))

In [23]:
X_train[0]

array([[1.        ],
       [0.11538462],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.33333333]])

In [24]:
y_train_1[0]

array([0.23076923])

In [25]:
# model = Sequential()

# model.add(Bidirectional(LSTM(100, 
